In [1]:
import pandas as pd 
import re
import nltk
nltk.download('punkt')
import numpy as np

import tensorflow as tf
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import LSTM ,SpatialDropout1D
from tensorflow.keras.layers import Bidirectional
from keras.layers import Dense
from tensorflow.keras.layers import Dropout
from sklearn import preprocessing


[nltk_data] Downloading package punkt to C:\Users\Muhammad
[nltk_data]     Swelam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import pandas as pd 
df = pd.read_csv("Arabic tweets classification project/clean_data.csv")
df=df.dropna()
df.reset_index(inplace=True)
df.head()

,index,id,txt,dialect
0,0,1175358310087892992,بالنهاية ينتفض يغير,IQ
1,1,1175416117793349632,يعني محسوب البشر حيونه ووحشيه وتطلبون الغرب يح...,IQ
2,2,1175450108898565888,مبين كلامه خليجي,IQ
3,3,1175471073770573824,يسلملي مرورك وروحك الحلوه,IQ
4,4,1175496913145217024,وين الغيبه اخ محمد,IQ


In [3]:
X = df['txt']
y = df['dialect']
voc_size=8000


In [4]:
label_encoder = preprocessing.LabelEncoder()
y= label_encoder.fit_transform(y)
y

array([4, 4, 4, ..., 1, 1, 1])

In [5]:
X

0                                       بالنهاية ينتفض يغير
1         يعني محسوب البشر حيونه ووحشيه وتطلبون الغرب يح...
2                                          مبين كلامه خليجي
3                                 يسلملي مرورك وروحك الحلوه
4                                        وين الغيبه اخ محمد
                                ...                        
457768              لين صبحنا بوجهش زين سعاده مابعدها سعاده
457769                                  بالعافيه فقلي الاكل
457770                                              مدح يبي
457771    ايييي احبها واحب تحتها طاوله للكرك ويفضل بلك ل...
457772                      جيبي مطرقه شوفي اهدم واهدم ابوه
Name: txt, Length: 457773, dtype: object

In [6]:
corpus=[]
for i in X:
    corpus.append(i)
    

onehot_repr=[one_hot(words,voc_size)for words in corpus] 


In [7]:
sent_length=30
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2338 2986  548]
 [   0    0    0 ... 7961  841 2452]
 [   0    0    0 ... 3700  212 1093]
 ...
 [   0    0    0 ...    0 1962 2659]
 [   0    0    0 ... 4060 5641 2548]
 [   0    0    0 ... 7733 4414  407]]


In [8]:
embedding_vector_features=50
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(18, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 50)            400000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 30, 50)           0         
 ropout1D)                                                       
                                                                 
 bidirectional (Bidirectiona  (None, 200)              120800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 18)                3618      
                                                                 
Total params: 524,418
Trainable params: 524,418
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
X_final=np.array(embedded_docs)
y_final=np.array(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)


In [10]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64)


Epoch 1/20
5723/5723 [==============================] - 352s 61ms/step - loss: 2.2346 - accuracy: 0.2976 - val_loss: 2.0649 - val_accuracy: 0.3543
Epoch 2/20
5723/5723 [==============================] - 351s 61ms/step - loss: 2.0254 - accuracy: 0.3681 - val_loss: 2.0029 - val_accuracy: 0.3762
Epoch 3/20
5723/5723 [==============================] - 365s 64ms/step - loss: 1.9572 - accuracy: 0.3898 - val_loss: 1.9849 - val_accuracy: 0.3827
Epoch 4/20
5723/5723 [==============================] - 357s 62ms/step - loss: 1.9130 - accuracy: 0.4047 - val_loss: 1.9701 - val_accuracy: 0.3880
Epoch 5/20
5723/5723 [==============================] - 365s 64ms/step - loss: 1.8793 - accuracy: 0.4157 - val_loss: 1.9689 - val_accuracy: 0.3910
Epoch 6/20
5723/5723 [==============================] - 377s 66ms/step - loss: 1.8523 - accuracy: 0.4241 - val_loss: 1.9642 - val_accuracy: 0.3909
Epoch 7/20
5723/5723 [==============================] - 418s 73ms/step - loss: 1.8291 - accuracy: 0.4309 - val_loss: 1

In [11]:
y_pred=model.predict(X_test)
y_pred=np.argmax(y_pred,axis=1)

y_pred=label_encoder.inverse_transform(y_pred)
y_test=label_encoder.inverse_transform(y_test)


from sklearn.metrics import confusion_matrix ,classification_report
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)

0.3876467697012725

In [12]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          AE       0.30      0.27      0.28      5330
          BH       0.24      0.18      0.20      5122
          DZ       0.45      0.37      0.40      3261
          EG       0.55      0.73      0.63     11591
          IQ       0.44      0.32      0.37      3050
          JO       0.29      0.23      0.26      5565
          KW       0.33      0.41      0.37      8414
          LB       0.47      0.49      0.48      5533
          LY       0.45      0.48      0.47      7353
          MA       0.50      0.45      0.47      2297
          OM       0.22      0.19      0.20      3858
          PL       0.36      0.41      0.38      8663
          QA       0.34      0.35      0.35      6198
          SA       0.25      0.26      0.25      5360
          SD       0.50      0.40      0.44      2887
          SY       0.31      0.22      0.26      3208
          TN       0.41      0.31      0.36      1834
          YE       0.18    